In [6]:
import ipfshttpclient
import requests
import json 
import fiona 
import geopandas
import pandas as pd
client = ipfshttpclient.connect()
import sqlite3
import hashlib
import Crypto
from Crypto.Cipher import AES
from base64 import b64encode, b64decode
import numpy as np
import time 

In [25]:
DB_Name = "bag_mini_testset_batch_upto5.gpkg"
username = "DWH-User"
organisation = "Org1"
key = "secret-key"

In [26]:
t = add_user(username, organisation)
tables_filtered = addDB(DB_Name)

There are: 3 tables
Containing: 3 batches


In [30]:
Confirm_integrity(DB_Name, tables_filtered, organisation, username)

Org1
se_bag_ligplaats
1
se_bag_ligplaats 1:
Org1
se_bag_ligplaats
2
se_bag_ligplaats 2:
Org1
se_bag_ligplaats
5
se_bag_ligplaats 5:
Org1
se_bag_woonplaats
1
se_bag_woonplaats 1:
Org1
se_bag_woonplaats
2
se_bag_woonplaats 2:
Org1
se_bag_woonplaats
3
se_bag_woonplaats 3:
Org1
se_bag_woonplaats
5
se_bag_woonplaats 5:
Integrity of: bag_mini_testset_batch_upto5.gpkg confirmed


()

In [3]:
#To load the dataframe

def get_tables(db):  #Get a list of the tables that are inside the database
    cursor = db.cursor()
    tables = list()
    for name in cursor.execute('SELECT * FROM gpkg_contents;'):
        tables.append(name[0])   
    cursor.close()
    return(tables)

def filter_tables(db, tables): #Filter for usefull tables, return a list of the useful tables. 
    tables_filterd = list()
    cursor = db.cursor()
    for table in tables:
        colNames = list() 
        for name in cursor.execute('PRAGMA table_info('+ table +');'):
            colNames.append(name[1])
        if "metahistoryinsbatchid" in colNames: #We only take tables that have the list metahistoryinsbatchid
            tables_filterd.append(table)        
    cursor.close() 
    return(tables_filterd) 


def get_amount_batches(tables_filtered, db):  #Get the numer of batched in the first col (BETTER TO LOOK AT AL DB"s AND SELECT the most)
    df = pd.read_sql_query("SELECT * from " + tables_filtered[0], db)
    return(df["metahistoryinsbatchid"].unique())
    

def addDB(File_name): #Add DB To the system 
    db = sqlite3.connect(File_name)
    tables = get_tables(db)
    tables_filtered = filter_tables(db, tables)
    print("There are: " + str(len(tables_filtered)) + " tables")
    print("Containing: "+ str(len(get_amount_batches(tables_filtered, db))) + " batches")
    db.close()
    return(tables_filtered)

#To add the dataframe

def add_user(username, organization): #add user using the API in hyperledger, returns the token needed to add transaction tot the chain. 
    URL ="http://localhost:4000/users"
    PARAMS = {'username':username, 
              'orgName': organization} 
    r = requests.post(url = URL, json = PARAMS)   
    token = json.loads(r.text)['token'] 
    return token


In [31]:
#Function to check the integrity, input is the database, what are the usefull tables and what organisation we are. It uses the identification attributes to retrieve the verification hash, 
#checks the verification hash with the verification hash on the blockchain. Only uses the verification attributes when the identificaion hash is wrong. 
def Confirm_integrity(file_name, tables_filtered, organisation, username):
    db = sqlite3.connect(file_name)
    final_reponds = 0
    for table_name in tables_filtered:
        table = get_table(table_name, db)
        for batch_number in table["metahistoryinsbatchid"].unique():
            subset = get_batch(table, batch_number)
            s_v, h_v = verification_attributes(subset, organisation, table_name)
            h_i_, h_v_= get_from_blockchain(organisation, table_name, batch_number)
            print(table_name + " " + str(batch_number) + ":")
            responds = verify_1(h_v, h_v_)
            if (responds == 1):
                final_reponds = 1
                verify_2(h_i_, s_v, key)
    if (final_reponds == 0):
        print('\x1b[1;32m'+'Integrity of: ' + file_name + ' confirmed' + '\x1b[0m')
        sent_certificate(organisation, file_name, username)
    else: 
        print('\x1b[1;31m'+'Integrity of: ' + file_name + ' NOT confirmed, see errors' + '\x1b[0m')
    db.close()
    return()



def sent_certificate(organisation, file_name, username):
    args = args = [organisation, file_name, username , time.strftime('%Y-%m-%d', time.localtime())]
    URL = "http://localhost:4000/channels/mychannel/chaincodes/IntegrityCertificate"
    PARAMS = {"fcn": "createRecord", "chaincodeName": "IntegrityCertificate", "channelName": "mychannel", "args" : args}
    headers = {"Authorization": "Bearer " + t}
    r = requests.post(url = URL, json = PARAMS, headers=headers)
    return()


def verification_attributes(data, organisation, table_name):
    batch_hash = [hashlib.sha256(repr(val).encode()).hexdigest() for val in data.T.apply(lambda x: (tuple(x)), axis = 1)]
    h_v = hashlib.sha256(repr("".join(batch_hash)).encode()).hexdigest()
    file = {
        'orgName': organisation,
        'tableName': table_name, 
        'batch': int(data["metahistoryinsbatchid"].unique()[0]),
        'colN': len(data.columns),
        'batchHash':h_v,
        'timeStamp': data["metahistoryvalidfrom"].unique()[0],
        'hashes': batch_hash,
        'cols:': list(data.columns) 
        }
    return(file, h_v)

#Compares the blockchain verification hash with the database verificaion hash.
def verify_1(h_v, h_v_):
    r = 0 
    if(str(h_v) == h_v_):
        k = 1 
    else:
        print("Integrity broken, investigate further")
        r = 1
    return(r)
 
    
#Compares the distributed identification hashes with the database identification hashes.     
def verify_2(h_i_, s_v, key): 
    file = get_from_ipfs(h_i_)
    secret = get_key_from_blockchain(h_i_)
    y = decrypt(key, secret, file)
    r = 0
    if(s_v['hashes'] == y['hashes'] ):
        k = 1 
    else:
        pair_compare(s_v, y)
        r = 1
    return(r)
  

def get_table(table_name, db):            
    return(pd.read_sql_query("SELECT * from " + table_name, db))     

def get_batch(data, batch_number):
    return(data.loc[data['metahistoryinsbatchid'] == batch_number]) 

#Retrieve the identification and verification hash from the blockchain. 
def get_from_blockchain(organisation, table_name, batch_number ):
    print(organisation)
    print(table_name)
    print(batch_number)
    URL = "http://localhost:5984/mychannel_$integrity_verification/_find"
    PARAMS = {"selector" : {"Organisation" : organisation, "Table_name" : table_name, "Batch_ID": str(batch_number)}}
    r = requests.post(url = URL, json = PARAMS)
    return(json.loads(r.text)['docs'][0]["_id"], json.loads(r.text)['docs'][0]['Verification_Hash']) 

def get_from_ipfs(h_i_):
    return(client.cat(h_i_))

def decrypt(key, nonce, file):
    cipher = AES.new(bytes(key, 'utf-8'), AES.MODE_EAX, b64decode(bytes(nonce[3:],'ISO-8859-1')))
    plaintext = cipher.decrypt(file)
    return(json.loads(plaintext.decode("utf-8")))

def get_key_from_blockchain(h_i): 
    URL = "http://localhost:4000/channels/mychannel/chaincodes/privateStorage"
    PARAMS = {"fcn": "readPrivateKey", "args" : """["collectionCarPrivateDetails" ,'""" +  h_i + "']"""}
    headers = {"Authorization": "Bearer " + t}
    result = None
    while result is None: 
        r = requests.get(url = URL, params = PARAMS, headers=headers)
        try:
            responds = json.loads(r.text)["result"]['secrets']
            return(responds)
        except:
            pass 
    return(responds) 
    

#Compare the distributed verification hashes with the datawarehouse verification hashes. 
def pair_compare(s_v, y):
    missing_col = list(set(s_v['cols:']) - set(y['cols:'])) + list(set(y['cols:']) - set(s_v['cols:'])) #search if there is a difference in columns. 
    if (len(missing_col) > 0): 
        print("Missing columns: ")
        print(missing_col)
 
    missing_hash = list(set(y['hashes']) - set(s_v['hashes'])) #search if there is a difference in hashes
    print(missing_hash)
    if (len(missing_hash) > 0): 
        print("Missing or wrong hash at column: ")
        for hash_ in missing_hash:
            n = y["hashes"].index(hash_)
            print(y['cols:'][n])
    return()


In [19]:
type(time.strftime('%Y-%m-%d', time.localtime()))

str